Moving Average:
$$ X_t = \mu + \varepsilon_t + \sum_{i=1}^q \theta_i \varepsilon_{t-i} $$

Auto-Regressive:
$$ X_t = c + \sum_{i=1}^p \varphi_i X_{t-i} + \varepsilon_t $$

Auto-Regressive Moving Average:
$$ X_t = \sum_{i=1}^p \varphi_i X_{t-i} + \varepsilon_t + \sum_{i=1}^q \theta_i \varepsilon_{t-i} $$

Auto-Regressive Integrated Moving Average:
<br>
<br>
From ARMA,
$$ X_t - \sum_{i=1}^{p^{\prime}} \varphi_i X_{t-i} = \varepsilon_t + \sum_{i=1}^q \theta_i \varepsilon_{t-i} $$
Let
$$ L^{i}X_t = X_{t-i}, i = 1, 2, 3... $$
and
$$ L^{i}\varepsilon_t = \varepsilon_{t-i}, i = 1, 2, 3... $$
then
$$ X_t - \sum_{i=1}^{p^{\prime}} \varphi_i L^{i}X_t = \varepsilon_t + \sum_{i=1}^q \theta_i L^{i}\varepsilon_t $$

$$ \left(1 - \sum_{i=1}^{p^{\prime}} \varphi_i L^{i}\right)X_t = \left(1 + \sum_{i=1}^q \theta_i L^{i}\right)\varepsilon_t $$

Assume that
$$ \left(1 - \sum_{i=1}^{p^{\prime}} \varphi_i L^{i}\right) = \left(1 - \sum_{i=1}^{p^{\prime} - d} \alpha_i L^{i}\right)(1 - L)^{d} $$

then
$$ \left(1 - \sum_{i=1}^{p} \alpha_i L^{i}\right)(1 - L)^{d}X_t = \left(1 + \sum_{i=1}^q \theta_i L^{i}\right)\varepsilon_t $$

where
$$ p = p^{\prime} - d \quad (\text{d integration}) $$